# Evaluación de PC-SMOTE con Grid Search en el dataset Shuttle (Generación de caso base y datasets aumentados)


In [ ]:
# lo que hace es modificar la lista de rutas de búsqueda de módulos de Python (sys.path) para incluir las carpetas ../scripts y ../datasets como ubicaciones adicionales donde Python puede buscar módulos o paquetes cuando hacés un import.
import sys
sys.path.append("../scripts")
sys.path.append("../datasets")

## Importación de módulos y librerías necesarias


In [ ]:
# --- Módulos propios del proyecto ---
from cargar_dataset import cargar_dataset                      # Función para cargar datasets según configuración
from config_datasets import config_datasets                    # Diccionario de configuración de datasets
from evaluacion import evaluar_sampler_holdout                 # Evaluación de sobremuestreo con partición hold-out
from custom_samplers import PCSMOTEWrapper                     # Wrapper personalizado para la técnica PCSMOTE
from pc_smote import PCSMOTE                                   # Implementación principal de PCSMOTE

# --- Librerías estándar de Python ---
from datetime import datetime, timedelta                       # Manejo de fechas y tiempos
from itertools import product                                  # Generación de combinaciones de parámetros
import os                                                      # Operaciones con el sistema de archivos

# --- Librerías científicas ---
import numpy as np                                              # Operaciones numéricas y algebra lineal
import pandas as pd                                             # Manipulación y análisis de datos tabulares
from scipy.stats import uniform                                 # Distribuciones para búsqueda de hiperparámetros

# --- Scikit-learn: preprocesamiento ---
from sklearn.preprocessing import LabelEncoder, StandardScaler # Codificación de etiquetas y escalado de datos
from sklearn.pipeline import make_pipeline, Pipeline            # Creación de pipelines de procesamiento y modelado

# --- Scikit-learn: división y validación ---
from sklearn.model_selection import (
    train_test_split,                                           # División de datos en train/test
    StratifiedKFold,                                            # Validación cruzada estratificada
    RandomizedSearchCV                                          # Búsqueda aleatoria de hiperparámetros
)

# --- Scikit-learn: reducción de dimensionalidad ---
from sklearn.decomposition import PCA                           # Análisis de Componentes Principales

# --- Scikit-learn: métricas ---
from sklearn.metrics import (
    f1_score,                                                    # Métrica F1-Score
    balanced_accuracy_score,                                     # Precisión balanceada
    matthews_corrcoef,                                           # Coeficiente MCC
    cohen_kappa_score,                                           # Kappa de Cohen
    make_scorer                                            
)

# --- Scikit-learn: clasificadores ---
from sklearn.ensemble import RandomForestClassifier             # Clasificador Random Forest
from sklearn.linear_model import LogisticRegression             # Regresión logística
from sklearn.svm import SVC                                      # Máquinas de Vectores de Soporte (SVM)

from sklearn.exceptions import ConvergenceWarning
import warnings

from conexion import DatabaseConnection  # tu clase
db = DatabaseConnection()
db.connect()
from GestorExperimentosDB import GestorExperimentosDB

gestor_db = GestorExperimentosDB(db) 

## Generación del caso base

Este código realiza dos tareas principales para cada dataset configurado en `config_datasets`:

1. **Generar el caso base** (subcarpeta `datasets_aumentados/base/`):
   - Se crea un directorio específico para almacenar la versión original del dataset sin ningún tipo de sobremuestreo.
   - El dataset se carga utilizando la misma función `cargar_dataset` empleada en el pipeline principal.
   - Si las etiquetas (`y`) están en formato de texto u objeto, se convierten a valores numéricos con `LabelEncoder`.
   - Se realiza una división estratificada en conjuntos de entrenamiento y prueba (`train/test`) utilizando `train_test_split` con una proporción 70/30 y una semilla fija para asegurar reproducibilidad.
   - Se guardan dos archivos CSV: `<nombre_dataset>_train.csv` y `<nombre_dataset>_test.csv`.

In [ ]:
# --- función: generar caso base (train/test sin sobremuestreo) ---

def generar_caso_base(
    nombre_dataset: str,
    config: dict,
    ruta_base: str = "../datasets/datasets_aumentados/base/",
    test_size: float = 0.30,
    random_state: int = 42,
    overwrite: bool = False
):
    """
    Genera el caso base (sin PCSMOTE) para un dataset: guarda train y test en ruta_base.
    Usa la misma lógica de carga que el resto del pipeline (cargar_dataset + LabelEncoder opcional).

    Retorna:
        (path_train, path_test)
    """
    os.makedirs(ruta_base, exist_ok=True)

    path_train = os.path.join(ruta_base, f"{nombre_dataset}_train.csv")
    path_test  = os.path.join(ruta_base, f"{nombre_dataset}_test.csv")

    if not overwrite and os.path.exists(path_train) and os.path.exists(path_test):
        return path_train, path_test  # ya generado

    # 1) Cargar dataset base con tu helper habitual
    X, y, _ = cargar_dataset(
        path=config["path"],
        clase_minoria=config.get("clase_minoria"),
        col_features=config.get("col_features"),
        col_target=config.get("col_target"),
        sep=config.get("sep", ","),
        header=config.get("header", None),
        binarizar=False,
        tipo=config.get("tipo", "tabular")
    )

    # 2) Asegurar etiquetas numéricas si vienen como strings/objects
    if getattr(y, "dtype", None) == object or (len(y) > 0 and isinstance(y[0], str)):
        y = LabelEncoder().fit_transform(y)

    # 3) Split estratificado (mismo seed para reproducibilidad)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )

    # 4) Guardar CSVs
    pd.concat([pd.DataFrame(X_train), pd.Series(y_train, name=config.get("col_target", "target"))], axis=1)\
      .to_csv(path_train, index=False)
    pd.concat([pd.DataFrame(X_test), pd.Series(y_test, name=config.get("col_target", "target"))], axis=1)\
      .to_csv(path_test, index=False)

    return path_train, path_test


## Configuración de Parámetros para PC-SMOTE

En esta sección, definimos los parámetros clave para la técnica PC-SMOTE:

- **percentiles_densidad**: Lista de percentiles (25, 50, 75) para controlar la densidad de vecindarios en el espacio de características.
- **percentiles_riesgo**: Lista de percentiles (25, 50, 75) para determinar el umbral de riesgo para la generación de muestras sintéticas.
- **criterios_pureza**: Diferentes criterios para evaluar la pureza de los clústeres durante el sobremuestreo.

Estos parámetros se utilizarán en una búsqueda en cuadrícula (grid search) para encontrar la mejor combinación que optimice el rendimiento del modelo en el dataset Shuttle.

### 🧬 Aumento de Datasets mediante Técnicas de Sobremuestreo

En esta etapa se genera una versión balanceada de cada dataset original mediante la aplicación de técnicas de sobremuestreo, con el objetivo de mitigar el desbalance de clases antes del entrenamiento de los modelos.

Actualmente, se emplea la técnica:

- `PCSMOTE` (Percentile-Controlled SMOTE), que permite controlar la generación de muestras sintéticas en función de percentiles de densidad, riesgo y pureza.

Para cada dataset, se exploran combinaciones específicas de parámetros según la técnica utilizada. Los datasets resultantes se almacenan en el directorio `datasets/datasets_aumentados/`, utilizando nombres de archivo que reflejan la configuración empleada (por ejemplo: `pcsmote_nombre_D25_R50_Pentropia_train.csv`).

> ⚠️ Esta fase no incluye entrenamiento ni validación de modelos. Su único propósito es generar conjuntos de datos aumentados a partir del conjunto de entrenamiento. La partición `train/test` se realiza previamente, y **solo la parte de entrenamiento es sometida a sobremuestreo**. El conjunto de prueba permanece sin modificar para garantizar una evaluación imparcial posterior.


In [ ]:
from time import time

percentiles_densidad = [25, 50, 75]
percentiles_riesgo   = [25, 50, 75]
criterios_pureza     = ["entropia", "proporcion"]

# Modelo "placeholder" cuando no entrenás nada acá (solo generás datasets)
# Si después entrenás LR/SVM/RF en otra notebook, ahí usarás su modelo_id real.
from time import time

# un "modelo" placeholder para esta notebook donde solo generás datasets
modelo_id = gestor_db.get_or_create_modelo_id("PCSMOTE_only")

for nombre_dataset, config in config_datasets.items():
    if nombre_dataset == "eurosat":
        continue

    print(f"\n📁 Dataset: {nombre_dataset}")

    for pdens in percentiles_densidad:
        for priesgo in percentiles_riesgo:
            for criterio in criterios_pureza:
                print(f"➡️  D={pdens} | R={priesgo} | P={criterio}")

                t0 = time()
                experimento_id, experimento_nombre, sampler = gestor_db.aumentar_dataset_pcsmote_y_guardar(
                    nombre_dataset=nombre_dataset,
                    config=config,
                    percentil_densidad=pdens,
                    percentil_riesgo=priesgo,
                    criterio_pureza=criterio
                )
                elapsed = round(time() - t0, 3)

                # Actualiza el mensaje de éxito
                if experimento_id and sampler:
                    print(f"✔ Guardado en DB. experimento_id={experimento_id} (elapsed={elapsed}s)")

                if not (experimento_id and sampler):
                    print("❌ Falló la generación.")
                    continue

                # 1) dataset_id (si no querés leer tamaños acá, dejá NULLs)
                n_train = None
                n_test  = None
                n_feat  = None
                dataset_id = gestor_db.get_or_create_dataset_id(nombre_dataset, n_train, n_test, n_feat, es_grande=0)

                # 2) config_id desde el sampler
                cfg = PCSMOTE.to_config_dict(sampler)
                config_id = gestor_db.get_or_create_config_id(**cfg)

                # 3) experimento + logs (usa sampler.guardar_en_db que ya agregaste)
                mejor_config = {
                    "pcsmote": {
                        "k": sampler.k,
                        "random_state": sampler._loggable_random_state(),
                        "percentil_densidad": sampler.percentil_densidad,
                        "percentil_riesgo": sampler.percentil_dist,
                        "criterio_pureza": sampler.criterio_pureza,
                        "percentil_entropia": sampler.percentil_entropia,
                        "modo_espacial": sampler.modo_espacial,
                        "factor_equilibrio": sampler.factor_equilibrio
                    }
                }

                experimento_id = sampler.guardar_en_db(
                    db,
                    dataset_id=dataset_id,
                    config_id=config_id,
                    modelo_id=modelo_id,
                    cv_splits=None, n_iter=None, n_jobs_search=None,
                    search_time_sec=elapsed,
                    mejor_configuracion=mejor_config,
                    source_file=None,
                    metricas=None,
                    guardar_logs=True,          # ✅ ahora guarda en `log_pcsmote`
                    tabla_logs="log_pcsmote"
                )

                print(f"✔ Guardado en DB. experimento_id={experimento_id} (elapsed={elapsed}s)")

db.close()